In [ ]:
# ------------ "in silico" reverse correlation experiment  ------------ 

# import needed modules
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import keras 

In [ ]:
# load model and display summary
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet')
print(base_model.summary())

In [ ]:
# generate and inspect noise (white) to be used in reverse correlation experiment
nframes=5000;
noise=np.random.uniform(0,1,[224,224,nframes]);
for j in range(5):
    print 'Noise frame number ',j,' ...'
    plt.imshow(noise[:,:,j])
    plt.show()

In [ ]:
# import needed functions
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [ ]:
# create sub-model to get activities at intermediate layers
from keras.models import Model
sub_model1=Model(input=base_model.input, output=base_model.get_layer('block1_conv1').output)
sub_model2=Model(input=base_model.input, output=base_model.get_layer('block2_conv1').output)
sub_model3=Model(input=base_model.input, output=base_model.get_layer('block5_conv1').output)

# decide from which units to "record"
nunit=10;
b1c1_units=np.empty([nunit,3])
b1c1_units[:,0]=np.random.randint(0,sub_model1.output_shape[1],nunit)
b1c1_units[:,1]=np.random.randint(0,sub_model1.output_shape[2],nunit)
b1c1_units[:,2]=np.random.randint(0,sub_model1.output_shape[3],nunit)
b2c1_units=np.empty([nunit,3])
b2c1_units[:,0]=np.random.randint(0,sub_model2.output_shape[1],nunit)
b2c1_units[:,1]=np.random.randint(0,sub_model2.output_shape[2],nunit)
b2c1_units[:,2]=np.random.randint(0,sub_model2.output_shape[3],nunit)
b5c1_units=np.empty([nunit,3])
b5c1_units[:,0]=np.random.randint(0,sub_model3.output_shape[1],nunit)
b5c1_units[:,1]=np.random.randint(0,sub_model3.output_shape[2],nunit)
b5c1_units[:,2]=np.random.randint(0,sub_model3.output_shape[3],nunit)

# initialize activity storage matrices
b1c1_activations = np.empty([nframes,nunit]);
b2c1_activations = np.empty([nframes,nunit]);
b5c1_activations = np.empty([nframes,nunit]);

# get activations
for j in range(nunit):
    for i in range(nframes):
        # load and preprocess current image
        img = np.dstack([noise[:,:,i],noise[:,:,i],noise[:,:,i]])
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # compute prediction and extract units activation
        b1c1_activations[i,j]=sub_model1.predict(x)[0,b1c1_units[j,0],
                                                    b1c1_units[j,1],
                                                    b1c1_units[j,2]]
        b2c1_activations[i,j]=sub_model2.predict(x)[0,b2c1_units[j,0],
                                                    b2c1_units[j,1],
                                                    b2c1_units[j,2]]
        b5c1_activations[i,j]=sub_model3.predict(x)[0,b5c1_units[j,0],
                                                    b5c1_units[j,1],
                                                    b5c1_units[j,2]]
    print 'activations of unit',j,'over every noise frame computed ...'
# save results
import pickle
f = open('VGG16activations.pckl', 'wb')
pickle.dump([b1c1_activations,b2c1_activations,b5c1_activations], f)
f.close()

In [ ]:
# do STA analysis %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
STAb1c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
STAb2c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
STAb5c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
for j in range(nunit):
    for i in range(nframes):
        STAb1c1[:,:,j]=STAb1c1[:,:,j]+b1c1_activations[i,j]*noise[:,:,i] 
        STAb2c1[:,:,j]=STAb2c1[:,:,j]+b2c1_activations[i,j]*noise[:,:,i] 
        STAb5c1[:,:,j]=STAb5c1[:,:,j]+b5c1_activations[i,j]*noise[:,:,i] 
    print 'STA analysis for unit',j,'in each layer completed ...'
STAb1c1=STAb1c1/nframes
STAb2c1=STAb2c1/nframes
STAb5c1=STAb5c1/nframes

# do STA analysis permutation test  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
nperm=25
STAb1c1P=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit,nperm])
STAb2c1P=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit,nperm])
STAb5c1P=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit,nperm])
for k in range(nperm):
    permidx=np.random.permutation(range(nframes))
    for j in range(nunit):
        for i in range(nframes):
            STAb1c1P[:,:,j,k]=STAb1c1P[:,:,j,k]+b1c1_activations[permidx[i],j]*noise[:,:,i] 
            STAb2c1P[:,:,j,k]=STAb2c1P[:,:,j,k]+b2c1_activations[permidx[i],j]*noise[:,:,i] 
            STAb5c1P[:,:,j,k]=STAb5c1P[:,:,j,k]+b5c1_activations[permidx[i],j]*noise[:,:,i] 
        print 'STA analysis permutation test number',k,'for unit',j,'in each layer completed ...'
STAb1c1P=STAb1c1P/nframes
STAb2c1P=STAb2c1P/nframes
STAb5c1P=STAb5c1P/nframes

sigSTAb1c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
sigSTAb2c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
sigSTAb5c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
muSTAb1c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
muSTAb2c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
muSTAb5c1=np.empty([noise[:,:,0].shape[0],noise[:,:,0].shape[1],nunit])
for j in range(nunit):
    sigSTAb1c1[:,:,j]=STAb1c1P[:,:,j,:].std(2)
    sigSTAb2c1[:,:,j]=STAb2c1P[:,:,j,:].std(2)
    sigSTAb5c1[:,:,j]=STAb5c1P[:,:,j,:].std(2)
    muSTAb1c1[:,:,j]=STAb1c1P[:,:,j,:].mean(2)
    muSTAb2c1[:,:,j]=STAb2c1P[:,:,j,:].mean(2)
    muSTAb5c1[:,:,j]=STAb5c1P[:,:,j,:].mean(2)
    
ZSTAb1c1 = (STAb1c1-muSTAb1c1)/sigSTAb1c1
ZSTAb2c1 = (STAb2c1-muSTAb2c1)/sigSTAb2c1
ZSTAb5c1 = (STAb5c1-muSTAb5c1)/sigSTAb5c1

In [ ]:
# print and save results
for j in range(nunit):
    print 'STA_unit_%d_block_1 ...' % j
    plt.imshow(ZSTAb1c1[:,:,j], clim=(-6,6))
    plt.colorbar()
    plt.savefig('STA_unit_%d_block_1.png' % j)
    plt.show()
    print 'STA_unit_%d_block_2 ...' % j
    plt.imshow(ZSTAb2c1[:,:,j], clim=(-6,6))
    plt.colorbar()
    plt.savefig('STA_unit_%d_block_2.png' % j)
    plt.show()
    print 'STA_unit_%d_block_5 ...' % j
    plt.imshow(ZSTAb5c1[:,:,j], clim=(-6,6))
    plt.colorbar()
    plt.savefig('STA_unit_%d_block_5.png' % j)
    plt.show()

In [ ]:
## how to load results
#f = open('VGG16activations.pckl', 'rb')
#prova = pickle.load(f)
#f.close()

In [ ]:
for j in range(nunit):
    for i in range(10):
        # load and preprocess current image
        img = np.dstack([noise[:,:,i],noise[:,:,i],noise[:,:,i]])
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # compute prediction and extract units activation
        value=sub_model1.predict(x)[0,b1c1_units[j,0],
                                                    b1c1_units[j,1],
                                                    b1c1_units[j,2]]
        print 'For unit',j,'and frame',i,'activation is',value